# Esercitazione 3: Teoria di Hanks

In questa esercitazione vedremo come implementare un sistema basato sulla teoria di Patrick Hanks. Nello specifico vedremo come generare i **semantic types** per le **collocations dei fillers** a partire da frasi estratte da un corpus.



L'approccio utilizzato si articola in 3 step principali:

* **Sub-corpus Extraction**: dato in input un **verbo target** ed un corpus, vengono estratte tutte le frasi del corpus in cui esso occorre. La ricerca viene implementata dalla funzione `hanks.corpus_extraction`.

* **Estrazione Fillers**: estrazione delle **collocations dei fillers** del verbo target.

* **Semantic Types**: i fillers estratti vengono **clusterizzati** in semantic-types.

Come risultato vengono visualizzate le collocations dei fillers e le informazioni di frequenza dei semantic types.


In [1]:
import nltk
from nltk.corpus import brown
import src.hanks as hanks
nltk.download('brown')
import spacy 

import random
random.seed(420)

[nltk_data] Downloading package brown to /home/prf/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
INPUT_VERB = 'eat'
VALENCE = 2

### Step 1: Sub-corpus Extraction

Dato in input un *target verb* ed un corpus, vengono estratte dal corpus tutte le frasi in cui esso occorre. La ricerca viene implementata dalla funzione `hanks.corpus_extraction`

Il corpus utilizzato in questa esercitazione è il *Brown Corpus* accessibile direttamente dalle API di NLTK.

In [3]:
selected_sents = hanks.corpus_extraction(INPUT_VERB, brown)
selected_sents[0]

['Most',
 'members',
 'of',
 'the',
 'U.S.',
 'Senate',
 ',',
 'because',
 'they',
 'are',
 'human',
 ',',
 'like',
 'to',
 'eat',
 'as',
 'high',
 'on',
 'the',
 'hog',
 'as',
 'they',
 'can',
 '.']

Le frasi del brown corpus sono in realtà tokenizzate e rappresentate come una sequenza di token. A questo proposito è possibile utilizzare il `TreebankWordDetokenizer` per effettuare l'operazione inversa.

In [4]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenizer = TreebankWordDetokenizer()
sents_sample = random.sample(selected_sents, 5)

for i, sent in enumerate(sents_sample, start=1):
    print(f"{i}. {detokenizer.detokenize(sent)}")


1. Why not put a cafe in each so the tourists would not have to travel too far to eat??
2. We haven't had anything to eat all day".
3. I picked him up, and the length of him arched very carefully and gracefully and only a little wildly, and I could feel the coolness of that radiant, fire-colored body, like splendid ice, and I knew that he had eaten only recently because there were two whole and solid little lumps in the forepart of him, like fieldmice swallowed whole might make.
4. they did not fall into pseudo-glamorous jobs on pseudo-glamorous magazines, but they did whatever nasty thing they could get in order to eat;;
5. If this woman had delayed until after 11:20 to start her shopping, she would have had little time in which to prepare the substantial meal that was eaten at dinner in those days.


Come si può osservare dall'output prodotto, in tutte le frasi compare il verbo "eat", si osservi come nella seconda e quinta frase, il verbo è nella forma past participle ("eaten").

### Step 2: Estrazione Fillers

dati in input un insieme di frasi, il verbo target e la **valenza** dei suoi argomenti, estrae tutti le **collocations** dei **fillers** presenti nelle frasi con cui gli argomenti si realizzano. Lo step viene implementato dalla funzione `hanks.find_verb_fillers`. 

Per realizzare questo step si necessita di effettuare il **parsing sintattico** della frase in input. Il parsing viene effettuato utilizzando la libreria Spacy. Costruendo il grafo delle dipendenze sintattiche è possibile recuperare una o più occorrenze del verbo/i target (funzione `hanks.find_target_verbs`) e successivamente, analizzando le relazioni sintattiche, estrarre i fillers utilizzando la funzione `hanks.get_hanks_verb`. 

La funzione `hanks.get_hanks_verb` ritorna una *named tuple* **HanksVerb** che rappresenta la realizzazione sintattica degli argomenti del verbo, così come suggerito dalla teoria stessa.

In [5]:
nlp = spacy.load("en_core_web_md")

fillers = hanks.find_verb_fillers(selected_sents, INPUT_VERB, nlp_pipeline=nlp, valence=2)

sent = fillers[4][0]
hanks_verb = fillers[4][1]

print(f"{detokenizer.detokenize(sent)}\n-->\n{hanks_verb}")

At the beginning of the school year, the new students don't eat the cereal right away, but within a short time they are eating it voraciously.
-->
HanksVerb(verb='eat', nargs=2, slot1='nsubj', slot2='dobj', filler1='students', filler2='cereal')


### Step 3: Semantic Type Clustering e WSD

In questo step, le informazioni estratte nel passo precedente e dunque le collocations dei fillers individuate, vengono clusterizzate in "cluster semantici" chiamati *semantic types*. 

La clusterizzazione utilizza  *WordNet* come sense-repository e dunque si necessita di un ulteriore step di **WSD**, effettuato dalla funzione `hanks.find_filler_senses` per disambiguare ed individuare il senso corretto associato al filler. Nell'esercitazione è stato utilizzato l'algoritmo **Lesk** con contesto di disambiguazione la frase stessa dove il filler del verbo occorre.

Dopo lo step di WSD, la clusterizzazione (funzione `hanks.semantic_clustering`) avviene considerando il **lexicographer file** (funzione `synset.lexname()`) a cui il senso associato al filler appartiene. Il lexicographer file rappresenta il **super-senso** del filler.  

In [6]:
import nltk.wsd as wsd

filler_senses = hanks.find_filler_senses(fillers, wsd.lesk)
for sense1, sense2 in filler_senses:
    print(f"{sense1} - {sense2}")

None - Synset('citizenry.n.01')
Synset('american_english.n.01') - Synset('fatty.a.01')
Synset('one.s.03') - Synset('skin.v.02')
None - None
Synset('scholar.n.01') - Synset('grain.n.02')
None - Synset('information_technology.n.01')
Synset('scholar.n.01') - Synset('grain.n.02')
None - Synset('information_technology.n.01')
Synset('one.n.01') - Synset('rider.n.01')
Synset('iodine.n.01') - Synset('testis.n.01')
None - Synset('seeded_player.n.01')
None - Synset('net_income.n.01')
Synset('world_health_organization.n.01') - Synset('celery.n.01')
Synset('one.s.01') - Synset('soup.n.02')
None - Synset('food.n.02')
Synset('helium.n.01') - Synset('nothing.r.01')
Synset('one.s.01') - Synset('more.r.02')
None - Synset('supper.n.01')
None - Synset('dinner.n.01')
None - Synset('information_technology.n.01')
Synset('fish.n.02') - Synset('torso.n.01')
None - Synset('one.s.01')
None - Synset('dust.v.02')
Synset('helium.n.01') - None
Synset('helium.n.01') - None
Synset('helium.n.01') - Synset('breakfast.v

Come si può osservare, la funzione `hanks.find_filler_senses` permette di passare dalla "word-form" del filler al "word-meaning", associando dunque un senso univoco.

Il risultato di questa fase dipende essenzialmente da due fattori:
* **Coverage** della risorsa lessicale utilizzata: "il senso espresso dal filler è presente in WordNet?" 
* **Performance** della funzione di WSD: a volte l'algoritmo lesk non riesce a trovare il senso wn da associare al filler (restituendo None).

In [7]:
def show_results(verb_name, fillers, filler_senses, semantic_types):
    print(f"Applying Hanks Theory for verb: {verb_name}")
    print()
    print("Collocations Extracted:\n")
    for  (_, hanks_verb), (filler1_sense, filler2_sense) in zip(fillers, filler_senses):
        print(f"filler: {hanks_verb.filler1} - {hanks_verb.filler2}")
        print(f"senses: {filler1_sense} - {filler2_sense}")
        print('-------------------------------------------------------------')
    print("")
    print(f"Semantic Types Clusters for {verb_name}:\n")
    tot = sum([freq for semantic_type, freq in semantic_types.most_common()])
    for semantic_type, freq in semantic_types.most_common():
        relative_freq = freq / tot * 100
        print(f"semantic type: {semantic_type}, occurence:{freq} ({relative_freq:0.2f}%)")
    print("******************************************************************\n\n")


In [8]:
semantic_types = hanks.semantic_clustering(filler_senses)
show_results(INPUT_VERB, fillers, filler_senses, semantic_types)

Applying Hanks Theory for verb: eat

Collocations Extracted:

filler: what - people
senses: None - Synset('citizenry.n.01')
-------------------------------------------------------------
filler: Americans - fat
senses: Synset('american_english.n.01') - Synset('fatty.a.01')
-------------------------------------------------------------
filler: one - skin
senses: Synset('one.s.03') - Synset('skin.v.02')
-------------------------------------------------------------
filler: what - they
senses: None - None
-------------------------------------------------------------
filler: students - cereal
senses: Synset('scholar.n.01') - Synset('grain.n.02')
-------------------------------------------------------------
filler: they - it
senses: None - Synset('information_technology.n.01')
-------------------------------------------------------------
filler: students - cereal
senses: Synset('scholar.n.01') - Synset('grain.n.02')
-------------------------------------------------------------
filler: they - i

### Risultati
Come si può notare dall'output, le prestazione sono abbastanza deludenti. Emergono due dinamiche:

* Spesso i filler corrispondenti all'argomento subject del verbo corrispondono a pronomi (I, she, they, ecc...) i quali non trovano una corrispondente lexical category in wordnet dunque lesk ritorna NONE , generando un semantic_type non-valido.

* Come si può notare l'"invalid semantic type" (None, None) rappresenta il 68% delle occorrenze, indicativo di come sia lo step di WSD che il coverage del sense repository (WN) ricoprando un ruolo fondamentale per il risultato finale.

Un importante osservazione da fare, che però non sorprende alla luce di quanto visto nel corso, è che lo step fondamentale che  ha l'impatto principale sull'intera pipeline è quello di **semantic-clustering**.

Per limitare il problema dell'invalid semantictype `(None, None)`, è possibile settare l'argomento `apply_custom_rule=True` nella funzione `hanks.find_filler_senses`. Questo argomento permette di applicare delle regole per "forzare" l'individuazione del senso associato a particolari fillers (ad es. per i pronomi "I", "you", ... vedasi la funzione `hanks.pronoun_WSD_rule`)

In [9]:
filler_senses = hanks.find_filler_senses(fillers, wsd.lesk, apply_custom_rule=True)
semantic_types = hanks.semantic_clustering(filler_senses)
show_results(INPUT_VERB, fillers, filler_senses, semantic_types)

Applying Hanks Theory for verb: eat

Collocations Extracted:

filler: what - people
senses: None - Synset('citizenry.n.01')
-------------------------------------------------------------
filler: Americans - fat
senses: Synset('american_english.n.01') - Synset('fatty.a.01')
-------------------------------------------------------------
filler: one - skin
senses: Synset('one.s.03') - Synset('skin.v.02')
-------------------------------------------------------------
filler: what - they
senses: None - None
-------------------------------------------------------------
filler: students - cereal
senses: Synset('scholar.n.01') - Synset('grain.n.02')
-------------------------------------------------------------
filler: they - it
senses: Synset('people.n.01') - Synset('information_technology.n.01')
-------------------------------------------------------------
filler: students - cereal
senses: Synset('scholar.n.01') - Synset('grain.n.02')
-------------------------------------------------------------

Confrontando questo output con quello precedente possiamo osservare che la frequenza dell' invalid semantic type si è (approssimativamente) dimezzata dall' $61\%$ al $35\%$ per effetto della regola applicata. 

### Batch Processing

In [10]:
from nltk.corpus import brown
from nltk.wsd import lesk

target_verbs = ['eat', 'meet', 'run', 'fight']
valence = 2

for target_verb in target_verbs:
    fillers, filler_senses, semantic_types = hanks.compute_hanks(target_verb, valence=2, corpus=brown, wsd_func=lesk, apply_rule = True)
    show_results(target_verb, fillers, filler_senses, semantic_types)

 occurence:1 (0.96%)
semantic type: ('noun.group', 'noun.quantity'), occurence:1 (0.96%)
semantic type: ('noun.artifact', 'noun.state'), occurence:1 (0.96%)
semantic type: ('noun.attribute', 'verb.communication'), occurence:1 (0.96%)
semantic type: ('noun.group', 'noun.cognition'), occurence:1 (0.96%)
semantic type: ('noun.substance', 'noun.act'), occurence:1 (0.96%)
semantic type: ('noun.group', 'noun.Tops'), occurence:1 (0.96%)
semantic type: ('verb.contact', 'noun.Tops'), occurence:1 (0.96%)
semantic type: ('noun.attribute', 'noun.Tops'), occurence:1 (0.96%)
semantic type: ('noun.object', 'verb.stative'), occurence:1 (0.96%)
semantic type: ('noun.person', 'noun.feeling'), occurence:1 (0.96%)
semantic type: ('noun.Tops', 'noun.group'), occurence:1 (0.96%)
semantic type: ('noun.cognition', 'noun.cognition'), occurence:1 (0.96%)
semantic type: ('noun.group', 'noun.possession'), occurence:1 (0.96%)
semantic type: ('adj.all', 'noun.location'), occurence:1 (0.96%)
semantic type: ('noun.fe